In [1]:
from datasets import load_dataset

dataset = load_dataset("coastalcph/lex_glue", "scotus")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1400
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1400
    })
})

In [4]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict


# Fit a TF-IDF vectorizer on the training texts ONLY
train_texts = dataset["train"]["text"]
vectorizer = TfidfVectorizer(lowercase=True)
vectorizer.fit(train_texts)

# Create a dictionary mapping each token to its IDF score
idf_dict = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))

def deduplicate_and_sort(text):
    """
    Remove duplicate tokens (preserving first occurrence) and then sort 
    the unique tokens in descending order according to their TF-IDF scores.
    
    Tokens not seen during training (and thus not in idf_dict) get a default score of 0.
    """
    # Tokenize the text using whitespace splitting; adjust if more advanced tokenization is needed.
    tokens = text.split()
    
    # Remove duplicates while preserving order
    seen = set()
    unique_tokens = []
    for token in tokens:
        # Convert to lowercase for checking against the TF-IDF dictionary
        lower_token = token.lower()
        if lower_token not in seen:
            seen.add(lower_token)
            unique_tokens.append(token)
    
    # Sort tokens by TF-IDF score in descending order. If a token isn't present, default to 0.
    sorted_tokens = sorted(unique_tokens, key=lambda t: idf_dict.get(t.lower(), 0), reverse=True)
    
    # Join back into a single string
    return " ".join(sorted_tokens)

def process_batch(example):
    """
    Process a batch of examples by applying deduplication and TF-IDF based sorting
    to the "text" field.
    """
    new_texts = [deduplicate_and_sort(t) for t in example["text"]]
    return {"text": new_texts}

# Map the processing function to all splits (train, validation, test if available)
new_dataset = dataset.map(process_batch, batched=True)

# (Optional) Check a sample from each split
print("Sample processed text from train split:")
print(new_dataset["train"][0]["text"])

if "validation" in new_dataset:
    print("\nSample processed text from validation split:")
    print(new_dataset["validation"][0]["text"])

if "test" in new_dataset:
    print("\nSample processed text from test split:")
    print(new_dataset["test"][0]["text"])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Sample processed text from train split:
Cimarron offerers Valvoline servitudes Enid racks arouses 19a mastery CHAMPLIN connects refinery Uncle charts tanks crosses converts maps understands subjection enclosed connect Sam tank crude tariffs indiscriminately departures unitary calculating pricing pipeline deliveries finished vitally inventory Originating continually carriage valuation differential generality disclaimed destination supplementary contemplation classify flows compiled remedied transmission wells gathering aforesaid purview stations Harry marketing premature commodity intrastate pipe lying producers REFINING convey departed fields partly transporting technically petroleum subordinate commodities treats owns exceeds falling transported artificial storage 1906 expenditures offend meaningless Valley copies Rock belonging schedule distributing truck hypothetical petitioned hire conventional practicable terminal Naturally road delegated manufacturing master expand greatest freig

In [9]:
def compute_lengt(new_dataset):
    highest=0
    total_length=0
    for idx in range(len(new_dataset['train'])):
        total_length+=len(new_dataset['train'][idx]['text'])
        if len(new_dataset['train'][idx]['text']) > highest:
            highest=len(new_dataset['train'][idx]['text'])
    print (f'The average length of documents in training dataset is {round(total_length/len(new_dataset['train']))}\nThe lengthiest document in the dataset contains {highest} number of tokens')        

In [10]:
compute_lengt(dataset)

The average length of documents in training dataset is 35723
The lengthiest document in the dataset contains 562772 number of tokens


In [11]:
compute_lengt(new_dataset)

The average length of documents in training dataset is 12137
The lengthiest document in the dataset contains 96425 number of tokens


In [12]:
new_dataset.push_to_hub("victorambrose11/lex_glue_TFIDF-SRT_variant_2")

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-67f63c5f-2fee6b17119c30b90ced4eb8;4c79767b-c182-482c-a053-47d3faa52eeb)

Invalid username or password.